# Material models
[source](http://www.fdtdxx.com/features/material-models)
[article](https://www.researchgate.net/publication/249651487_Free-Space_Transmission_Method_for_the_Characterization_of_Dielectric_and_Magnetic_Materials_at_Microwave_Frequencies)

In [ ]:
using DDA, Plots
gr();

## Dispersive Materials

Dispersive (frequency-dependent) materials can correctly describe the physics behind the optical responses. 

- Dielectric constant of free electron gas: 
  $$\varepsilon (\omega) = 1 - \frac{\omega_p^2}{\omega^2}$$
  
- Drude model: describes intraband electron motion.
  $$\varepsilon (\omega) = 1 - \frac{\omega_p^2}{\omega^2 + j \omega \gamma}$$

- Lorentz oscillator model: describes interband electron transitions.
  $$\varepsilon (\omega) = 1 + \frac{\Delta\varepsilon_p \omega_p^2}{\omega_p^2 - \omega(\omega + j 2\delta)}$$

- Debye model: describes the relaxation of atomic vibrations.
  $$\varepsilon (\omega) = \varepsilon_\infty  + \frac{\Delta\varepsilon}{1 + j \omega \tau}$$
  where 
  - $\Delta\varepsilon =  \varepsilon_\infty - \varepsilon_s$
  - $\tau = \frac{1}{\omega_r}$
  - $\varepsilon_s$ is a static dielectric permittivity,
  - $\varepsilon_\infty$ is permittivity at infinite frequency (optical permittivity), 
  - $\omega_r$ is relaxation frequency.

- Cole–Cole model 
  $$\varepsilon (\omega) = \varepsilon_\infty  + \frac{\Delta\varepsilon}{1 + j (\omega \tau)^{(1-\alpha)}} $$

- Cole-Davidson
  $$\varepsilon (\omega) = \varepsilon_\infty  + \frac{\Delta\varepsilon}{(1 + j \omega \tau)^\beta} $$
  
- Havriliak-Negami
  $$\varepsilon (\omega) = \varepsilon_\infty  + \frac{\Delta\varepsilon}{(1 + j (\omega \tau)^{(1-\alpha)})^\beta} $$

The terms $\alpha$ and $\beta$ are empirical parameters and their values are between $0$ and $1$. 
- $\alpha$ is a damping factor (broaden the dispersion) and describes the degree of flatness of the relaxation region. 
- $\beta$ is an asymmetric factor and describes relaxation properties asymmetric around relaxation frequency

$$ \varepsilon(\omega) = 1 - \frac{\omega_p^2}{\omega^2 + j \omega \gamma} =  1 - \frac{\omega_p^2}{\omega^2 + j \omega \gamma} +  1 - \frac{\omega_p^2}{\omega^2 + j \omega \gamma}  $$

## Arbitrarily-accurate Models

The arbitrarily-accurate (albeit, perhaps unphysical) fits to experimental optical data can be obtained.

- multi-pole Debye model with static conductivity
  $$\varepsilon (\omega) = \varepsilon_\infty  + \sum \limits_i^N \frac{\Delta\varepsilon_i}{1 + j \omega \tau_i} + \frac{\sigma_s}{j \omega \varepsilon_0}$$
  where
  - $\sigma_s$ the static conductivity
  - $\Delta\varepsilon_i$ is the change in permittivity due to the ith dispersion
- Drude model in combination with multiple Lorentz oscillator poles
  $$\varepsilon (\omega) = \varepsilon_\infty  + \sum \limits_N \frac{\Delta\varepsilon_p \omega_p^2}{\omega_p^2 - \omega(\omega + j 2\delta)}$$
- Interpolated tabulated data


## Nonlocal Materials

Spatially-nonlocal effects arise when electrons are confined to small regions of space. Examples include junctions between structures, the areas around the edges of small structures, etc. These can significantly modify the electromagnetic fields, relative to classical (local) effects.


In [ ]:
# Permittivity
# Electric susceptibility
# import Base: size, length

abstract type AbstractPermittivity end
abstract type DielectricMaterials <: AbstractPermittivity end
abstract type DispersiveMaterials <: AbstractPermittivity end
abstract type Tabulated <: AbstractPermittivity end


Broadcast.broadcastable(m::AbstractPermittivity) = Ref(m)
# https://github.com/JuliaLang/julia/pull/35591#issuecomment-619272218

const esp_0 = 1.


## Dielectric (Conductor) Materials

- dielectric constant:
  $$\varepsilon (\omega) = \varepsilon$$

In [ ]:
# # complex (real or complex)
struct DielectricConstant{T<:Number}  <: DielectricMaterials
    eps::T
end

permittivity(m::DielectricConstant) = m.eps
permittivity(m::DielectricConstant, omega) = permittivity(m)

In [ ]:
omega = 0:.1:10
model = DielectricConstant(1.)

ε = permittivity.(model, omega)

plot(omega, real(ε), label="Re(ε(ω))")

- dielectric model: This includes perfect (lossless) dielectrics, conductive (lossy) materials, and perfect electric conductors.
$$\varepsilon (\omega) = \varepsilon_\infty + \frac{\sigma}{j \omega \varepsilon_0}$$
$$\varepsilon (\omega) = \varepsilon_\infty + \frac{\sigma}{j \omega} = \varepsilon_\infty -j \frac{\sigma}{\omega}$$


In [ ]:
struct DielectricModel{T<:AbstractFloat} <: DielectricMaterials
    eps_inf::T
    sigma::T
end

permittivity(m::DielectricModel, omega) = m.eps_inf + m.sigma / (im *  omega * esp_0)


In [ ]:
omega = 0:.1:10
model = DielectricModel(1., 1.)

ε = permittivity.(model, omega)

# plot(omega, [real(ε) imag(ε)], label=["Re(ε(ω))" "Im(ε(ω))"], layout=(2,1))
p1 = plot(omega, real(ε), label="Re(ε(ω))")
p2 = plot(omega, imag(ε), label="Im(ε(ω))", legend=:bottomright)
plot(p1, p2, layout = @layout [a; b])

In [ ]:
model

## Dispersive Materials

Dispersive (frequency-dependent) materials can correctly describe the physics behind the optical responses. 

- Debye model: describes the relaxation of atomic vibrations.
  $$\varepsilon (\omega) = \varepsilon_\infty  + \frac{\Delta\varepsilon}{1 + j \omega \tau}$$
  where 
  - $\Delta\varepsilon =  \varepsilon_\infty - \varepsilon_s$
  - $\varepsilon_s$ is a static (low frequency) dielectric permittivity,
  - $\varepsilon_\infty$ is permittivity at high frequency limit (optical permittivity), 
  - $\tau = \frac{1}{\omega_r}$ is the characteristic relaxation time,
  - $\omega_r$ is relaxation frequency.

In [ ]:
struct DebyeModel{T<:AbstractFloat} <: DispersiveMaterials
    eps_s::T
    eps_inf::T
    tau::T
end

# TODO: minus or plus sign at (1 + - im * omega * m.tau)
permittivity(m::DebyeModel, omega) = m.eps_inf + (eps_s - eps_inf) / (1 - im * omega * m.tau);


In [ ]:
# omega = 0:.1:10
omega = exp10.(-2:.1:2)
eps_s = 6.
eps_inf = 0.
tau = 1.


m = DebyeModel(eps_s, eps_inf, tau)
ε = permittivity.(m, omega)

plot(omega, [real(ε) imag(ε)], label=["Re(ε(ω))" "Im(ε(ω))"], layout=(2, 1) ,)


# p1 = plot(omega, real(ε), label="Re(ε(ω))")
# p2 = plot(omega, imag(ε), label="Im(ε(ω))")
p1 = plot(omega, real(ε), label="Re(ε(ω))", xaxis=:log)
p2 = plot(omega, imag(ε), label="Im(ε(ω))", xaxis=:log)
plot(p1, p2, layout = @layout [a; b])

Variants of the Debye equation:

- Cole–Cole model 
  $$\varepsilon (\omega) = \varepsilon_\infty  + \frac{\Delta\varepsilon}{1 + j (\omega \tau)^{(1-\alpha)}} $$

- Cole-Davidson
  $$\varepsilon (\omega) = \varepsilon_\infty  + \frac{\Delta\varepsilon}{(1 + j \omega \tau)^\beta} $$
  
- Havriliak-Negami
  $$\varepsilon (\omega) = \varepsilon_\infty  + \frac{\Delta\varepsilon}{(1 + j (\omega \tau)^{(1-\alpha)})^\beta} $$

The terms $\alpha$ and $\beta$ are empirical parameters and their values are between $0$ and $1$. 
- $\alpha$ is a damping factor (broaden the dispersion) and describes the degree of flatness of the relaxation region. 
- $\beta$ is an asymmetric factor and describes relaxation properties asymmetric around relaxation frequency

- Dielectric constant of free electron plasma: 
  $$\varepsilon (\omega) = 1 - \frac{\omega_p^2}{\omega^2}$$

In [ ]:
struct FreeElectronPlasmaModel <: DispersiveMaterials
    omega_p
end

permittivity(m::FreeElectronPlasmaModel, omega) = 1 - m.omega_p^2 / omega^2;

In [ ]:
omega = 0:.1:10
omega_p = 3.
gamma = 1.

m = FreeElectronPlasmaModel(omega_p)
ε = permittivity.(m, omega)

plot(omega, ε, label="Re(ε(ω))", ylimits=[-5, 1], legend=:bottomright)

- Drude model: describes intraband electron motion.
  $$\varepsilon (\omega) = 1 - \frac{\omega_p^2}{\omega^2 + j \omega \gamma}$$

  note: 
  - $\gamma = \Gamma = \tau^{-1}$
  - Lorentz mode wit $\omega_0=0$

In [ ]:
struct DrudeModel{T<:AbstractFloat} <: DispersiveMaterials
    omega_p::T
    gamma::T
end

permittivity(m::DrudeModel, omega) = 1 - m.omega_p^2 / (omega^2 + im * m.gamma * omega);
# static low frequency
offset(m::DrudeModel) = 1 - m.omega_p^2 / m.gamma^2
cross(m::DrudeModel) = sqrt(m.omega_p^2 - m.gamma^2)
peak(m::DrudeModel) = Inf

In [ ]:
omega = 0:.01:10
omega_p = 3.
gamma = 1.

m = DrudeModel(omega_p, gamma)
ε = permittivity.(m, omega)

# plot(omega, [real(ε) imag(ε)], label=["Re(ε(ω))" "Im(ε(ω))"], layout=(2,1) )
p1 = plot(omega, real(ε), label="Re(ε(ω))", legend=:bottomright)
p2 = plot(omega, imag(ε), label="Im(ε(ω))")
plot(p1, p2, layout = @layout [a; b])

In [ ]:
offset(m), cross(m)

In [ ]:
m_free = FreeElectronPlasmaModel(omega_p)
ε_free = permittivity.(m_free, omega)

plot(omega, real(ε), label="Drude model", ylimits=[-10, 1])
plot!(omega, ε_free, label="Free electron plasma", legend=:bottomright)


- Lorentz oscillator model: describes interband electron transitions.
  
  $$\varepsilon (\omega) = 1 + \frac{\Delta\varepsilon_p \omega_p^2}{\omega_p^2 - \omega(\omega + j 2\delta)}$$
  ??

  $$\varepsilon (\omega) = 1 + \frac{\omega_p^2}{\omega_0^2  - \omega^2 - j \omega\Gamma}$$
  $$\varepsilon (\omega) = \varepsilon_\infty + \frac{(\varepsilon_s - \varepsilon_\infty) \omega_0^2}{\omega_0^2  - \omega^2 - j \omega\Gamma}$$


- The Cauchy distribution has the probability density function (PDF):
  $$ 
  f(x; x_0,\gamma) = \frac{1}{\pi\gamma \left[1 + \left(\frac{x - x_0}{\gamma}\right)^2\right]} = { 1 \over \pi \gamma } \left[ { \gamma^2 \over (x - x_0)^2 + \gamma^2  } \right], 
  $$
  where 
  - $x_0$ is the location parameter, specifying the location of the peak of the distribution, 
  - $\gamma$ is the scale parameter which specifies the half-width at half-maximum (HWHM), alternatively $2\gamma$ is full width at half maximum (FWHM).


In [ ]:
struct LorentzModel{T<:AbstractFloat} <: DispersiveMaterials
    omega_p::T
    omega_0::T
    gamma::T
end

permittivity(m::LorentzModel, omega) = 1 + m.omega_p^2 / (m.omega_0^2 - omega^2 - im * omega * m.gamma);

# static ε (low frequency)
offset_static(m::LorentzModel) = 1 + m.omega_p^2 / m.omega_0^2

# ε at high frequency
offset_inf(m::LorentzModel) = 1

# Absorption peak (imaginary part)
peak(m::LorentzModel) = m.omega_p^2 / (m.omega_0 * m.gamma)

In [ ]:
omega = 0:.01:4
omega_p = 2.
omega_0 = 2.
gamma = .2

m = LorentzModel(omega_p, omega_0,  gamma)
ε = permittivity.(m, omega)

@show offset_static(m), offset_inf(m), peak(m)
@show real(ε)[1], real(ε)[end], maximum(imag(ε))

p1 = plot(omega, real(ε), label="Re(ε(ω))")
p2 = plot(omega, imag(ε), label="Im(ε(ω))")
plot(p1, p2, layout = @layout [a; b])

In [ ]:
struct LorentzMode3{T<:AbstractFloat} <: DispersiveMaterials
    omega_p::T
    omega_0::T
    gamma::T
end

permittivity(m::LorentzMode3, omega) = m.omega_p^2 / (m.omega_0^2 - omega^2 - im * omega * m.gamma);

# static ε (low frequency)
offset_static(m::LorentzMode3) = m.omega_p^2 / m.omega_0^2

# ε at high frequency
offset_inf(m::LorentzMode3) = 0

# Absorption peak (imaginary part)
peak(m::LorentzMode3) = m.omega_p^2 / (m.omega_0 * m.gamma)

In [ ]:
omega = 0:.01:4
omega_p = 2.
omega_0 = 1.
gamma = .4

m = LorentzMode3(omega_p, omega_0,  gamma)
ε = permittivity.(m, omega)

@show offset_static(m), offset_inf(m), peak(m)
@show real(ε)[1], real(ε)[end], maximum(imag(ε))

p1 = plot(omega, real(ε), label="Re(ε(ω))")
p2 = plot(omega, imag(ε), label="Im(ε(ω))")
plot(p1, p2, layout = @layout [a; b])

In [ ]:
struct LorentzModel2{T<:AbstractFloat} <: DispersiveMaterials
    eps_s::T
    eps_inf::T
    omega_0::T
    gamma::T
end

permittivity(m::LorentzModel2, omega) =  m.eps_inf + (m.eps_s - m.eps_inf) * m.omega_0^2 / (m.omega_0^2 - omega^2 - im * omega * m.gamma);

# static ε (low frequency)
offset_static(m::LorentzModel2) = m.eps_s

# ε at high frequency
offset_inf(m::LorentzModel2) = m.eps_inf

# Absorption peak (imaginary part)
peak(m::LorentzModel2) = (m.eps_s - m.eps_inf) * m.omega_0 /  m.gamma



In [ ]:
omega = 0:.01:4
eps_s = 4.
eps_inf = 0.
omega_0 = 2.
gamma = .4

m = LorentzModel2(eps_s, eps_inf, omega_0,  gamma)
ε = permittivity.(m, omega)

@show offset_static(m), offset_inf(m), peak(m)
@show real(ε)[1], real(ε)[end], maximum(imag(ε))

p1 = plot(omega, real(ε), label="Re(ε(ω))")
p2 = plot(omega, imag(ε), label="Im(ε(ω))")
plot(p1, p2, layout = @layout [a; b])

In [ ]:
struct LorentzMode{T<:AbstractFloat} <: DispersiveMaterials
    eps_s::T
    omega_0::T
    gamma::T
end

permittivity(m::LorentzMode, omega) =   m.eps_s * m.omega_0^2 / (m.omega_0^2 - omega^2 - im * omega * m.gamma);

# static low frequency
offset(m::LorentzMode) = m.eps_s
peak(m::LorentzMode) = m.eps_s * m.omega_0 /  m.gamma


# static ε (low frequency)
offset_static(m::LorentzMode) = m.eps_s

# ε at high frequency
offset_inf(m::LorentzMode) = 0

# Absorption peak (imaginary part)
peak(m::LorentzMode) =  m.eps_s * m.omega_0 /  m.gamma

# LorentzMode2(eps_s, omega_0, gamma) = LorentzModel2(eps_s, 0., omega_0,  gamma)

In [ ]:
struct LorentzModel2{T<:AbstractFloat} <: DispersiveMaterials
    eps_s::T
    eps_inf::T
    omega_0::T
    gamma::T
end

permittivity(m::LorentzModel2, omega) =  m.eps_inf + (m.eps_s - m.eps_inf) * m.omega_0^2 / (m.omega_0^2 - omega^2 - im * omega * m.gamma);

# static ε (low frequency)
offset_static(m::LorentzModel2) = m.eps_s

# ε at high frequency
offset_inf(m::LorentzModel2) = m.eps_inf

# Absorption peak (imaginary part)
peak(m::LorentzModel2) = (m.eps_s - m.eps_inf) * m.omega_0 /  m.gamma



In [ ]:
omega = 0:.01:4
eps_s = 4.
eps_inf = 0.
omega_0 = 2.
gamma = .4

m = LorentzModel2(eps_s, eps_inf, omega_0,  gamma)
ε = permittivity.(m, omega)

@show offset_static(m), offset_inf(m), peak(m)
@show real(ε)[1], real(ε)[end], maximum(imag(ε))

p1 = plot(omega, real(ε), label="Re(ε(ω))")
p2 = plot(omega, imag(ε), label="Im(ε(ω))")
plot(p1, p2, layout = @layout [a; b])

In [ ]:
struct LorentzMode{T<:AbstractFloat} <: DispersiveMaterials
    eps_s::T
    omega_0::T
    gamma::T
end

permittivity(m::LorentzMode, omega) =   m.eps_s * m.omega_0^2 / (m.omega_0^2 - omega^2 - im * omega * m.gamma);

# static low frequency
offset(m::LorentzMode) = m.eps_s
peak(m::LorentzMode) = m.eps_s * m.omega_0 /  m.gamma


# static ε (low frequency)
offset_static(m::LorentzMode) = m.eps_s

# ε at high frequency
offset_inf(m::LorentzMode) = 0

# Absorption peak (imaginary part)
peak(m::LorentzMode) =  m.eps_s * m.omega_0 /  m.gamma

# LorentzMode2(eps_s, omega_0, gamma) = LorentzModel2(eps_s, 0., omega_0,  gamma)

In [ ]:
omega = 0:.01:4
eps_s = 1.
omega_0 = 2.
gamma = .2

m = LorentzMode2(eps_s, omega_0,  gamma)
ε = permittivity.(m, omega)

@show typeof(m)
@show offset_static(m), offset_inf(m), peak(m)
@show real(ε)[1], real(ε)[end], maximum(imag(ε))

p1 = plot(omega, real(ε), label="Re(ε(ω))")
p2 = plot(omega, imag(ε), label="Im(ε(ω))")
plot(p1, p2, layout = @layout [a; b])


- Lorentz oscillator model: describes interband electron transitions.
  $$\varepsilon (\omega) = 1 + \frac{\Delta\varepsilon_p \omega_p^2}{\omega_p^2 - \omega(\omega + j 2\delta)}$$
  $$\varepsilon (\omega) = 1 + \frac{\omega_p^2}{\omega_0^2  - \omega^2 + j \omega\Gamma}$$


# Tabulated 

In [ ]:



# Interpolated frequency dependent permitivirty
struct PermittivityTable <: TablePermittivity
    lambda::Vector{Float64}
    eps::Vector{ComplexF64}
    itp_real::Spline1D
    itp_imag::Spline1D
    function PermittivityTable(lambda::Vector{Float64}, eps::Vector{ComplexF64})

        inds = sortperm(lambda)
        # itp_real = interpolate((lambda[inds],), real(eps[inds]), Gridded(Linear()))
        # itp_imag = interpolate((lambda[inds],), imag(eps[inds]), Gridded(Linear()))

        itp_real = Spline1D(lambda[inds], real(eps[inds]))
        itp_imag = Spline1D(lambda[inds], imag(eps[inds]))

        new(lambda, eps, itp_real, itp_imag)
    end
end

permittivity(e::PermittivityTable, lambda) = e.itp_real(lambda) + e.itp_imag(lambda) * im

# TODO: proper broadcasting
reflective_index(e::AbstractPermittivity, lambda) = sqrt(permittivity(e, lambda))


# Constant permitivity

In [ ]:
m1 = DielectricConstant(2.)
m1, isapprox(permittivity(m1), permittivity(m1, 23523.235))

In [ ]:
m2 = DielectricConstant(-0.09850 + 1.9392im)
m2, isapprox(permittivity(m2), -0.09850 + 1.9392im)

# Wavelength dependent permitivity of gold

In [ ]:
# Gold, evaporated (Johnson & Christy 1972, PRB 6, 4370)
# wave(um)
eps = [
    -189.0 + 25.36im,
    -125.4 + 12.56im,
    -90.43 + 8.19im,
    -66.22 + 5.70im,
    -51.05 + 3.86im,
    -40.27 + 2.79im,
    -32.04 + 1.93im,
    -25.81 + 1.63im,
    -20.61 + 1.27im,
    -16.82 + 1.07im,
    -13.65 + 1.04im,
    -10.66 + 1.37im,
    -8.11 + 1.66im,
    -5.84 + 2.11im,
    -3.95 + 2.58im,
    -2.28 + 3.81im,
    -1.70 + 4.84im,
    -1.76 + 5.28im,
    -1.69 + 5.65im,
    -1.70 + 5.72im,
    -1.65 + 5.74im,
    -1.60 + 5.64im,
    -1.40 + 5.61im,
    -1.23 + 5.60im,
    -1.31 + 5.54im,
    -1.36 + 5.57im,
    -1.23 + 5.85im,
    -1.24 + 5.79im,
    -1.23 + 5.78im,
    -1.31 + 5.60im,
    -1.33 + 5.49im,
    -1.37 + 5.28im,
    -1.35 + 4.98im,
    -1.24 + 4.72im,
    -1.08 + 4.49im,
    -0.89 + 4.34im,
    -0.74 + 4.16im,
    -0.62 + 4.06im,
    -0.55 + 3.89im,
    -0.42 + 3.83im,
    -0.35 + 3.71im,
    -0.23 + 3.61im,
    -0.13 + 3.51im,
    -0.01 + 3.39im,
    0.14 + 3.40im,
    0.20 + 3.33im,
    0.29 + 3.29im,
    0.30 + 3.18im,
    0.23 + 3.04im
]

lambda = [
    1.937,
    1.610,
    1.393,
    1.216,
    1.088,
    0.9840,
    0.8920,
    0.8211,
    0.7560,
    0.7045,
    0.6595,
    0.6168,
    0.5821,
    0.5486,
    0.5209,
    0.4959,
    0.4714,
    0.4509,
    0.4305,
    0.4133,
    0.3974,
    0.3815,
    0.3679,
    0.3542,
    0.3425,
    0.3315,
    0.3204,
    0.3107,
    0.3009,
    0.2924,
    0.2844,
    0.2761,
    0.2689,
    0.2616,
    0.2551,
    0.2490,
    0.2426,
    0.2371,
    0.2313,
    0.2262,
    0.2214,
    0.2164,
    0.2119,
    0.2073,
    0.2033,
    0.1993,
    0.1953,
    0.1916,
    0.1879
]



In [ ]:
m = PermittivityTable(lambda, eps)

l = LinRange(minimum(lambda), maximum(lambda), 1000)
# l = LinRange(0., maximum(lambda), 1000)
e = permittivity(m, l)
size(e)

In [ ]:
# scatter(lambda, real(eps), xaxis=:log)
scatter(lambda, real(eps))
plot!(l, real(e))

In [ ]:
# scatter(lambda, imag(eps), xaxis=:log)
scatter(lambda, imag(eps))
plot!(l, imag(e))

# Appendix

## Drude model: Driven harmonic oscillator

source: [Plasmonics: Fundamentals and Applications](https://link.springer.com/book/10.1007/0-387-37825-1)

One can write a simple equation of motion for an electron of the plasma sea subjected to an external electric field $E$:
$$
m \ddot{x} + m \gamma \dot{x} = −e E
$$

If we assume a harmonic time dependence $E(t) = E_0 e^{−i \omega t}$ of the driving field, a particular solution of this equation describing the oscillation of the electron is $x(t) = x_0 e^{−i \omega t}$. The complex amplitude $x_0$ incorporates any phase shifts between driving field and response via
$$
x(t) = \frac{e}{m (\omega^2 + i \gamma \omega)} E(t).
$$

The displaced electrons contribute to the macroscopic polarization $P = −nex$, explicitly given by
$$
P = -\frac{n e^2}{m(\omega^2 + i \gamma \omega)} E.
$$

Inserting this expression for $P$ into equation ($D = \varepsilon_0 E + P$) yields
$$
D = \varepsilon_0(1 - \frac{\omega_p^2}{\omega^2 + i \gamma \omega})E,
$$
where $\omega_p^2 = \frac{n e^2}{\varepsilon_0 m}$ is the plasma frequency of the free electron gas. Therefore we arrive at the desired result, the dielectric function of the free electron gas:
$$
\varepsilon(\omega) = 1 - \frac{\omega_p^2}{\omega^2 + i \gamma \omega}.
$$
The real and imaginary components of this complex dielectric function $\varepsilon(\omega) = \varepsilon_1(\omega) + i \varepsilon_2(\omega)$ are given by
$$
\varepsilon_1(\omega) = 1 - \frac{\omega_p^2 \tau^2}{1 + \omega^2 \tau^2}.
$$
$$
\varepsilon_2(\omega) = 1 - \frac{\omega_p^2 \tau}{\omega(1 + \omega^2 \tau^2)}.
$$
where we have used $\gamma = \frac{1}{\tau}$.

## Free electron gas

It is insightful to study (???) for a variety of different frequency regimes with respect to the collision frequency $\gamma$. We will limit ourselves here to frequencies $\omega < \omega_p$, where metals retain their metallic character. For large frequencies close to $\omega_p$ , the product $\omega \tau \gg 1$, leading to negligible damping. Here, $\varepsilon(\omega)$ is predominantly real, and
$$
\varepsilon(\omega) = 1- \frac{\omega_p^2}{\omega^2}
$$

can be taken as the dielectric function of the undamped free electron plasma.


# Lorentz modes: Driven harmonic oscillators with dampening

Driven harmonic oscillators are damped oscillators further affected by an externally applied sinusoidal driving force:
$$
\ddot{x}(t) + \gamma \dot{x}(t)  + \omega_0^2 x(t) = A_0 e^{-i \omega t}
$$
where
- $A_0$ is the driving amplitude,
- $\omega$ is the driving frequency,
- $\omega_{0}$ is the undamped angular frequency,
- $\gamma$ is the damping ratio

Lest suppose that the solution exist in the following form ($\gamma > 0$):

$$
x(t) = x_0 e^{-i \omega_0 t} e^{-\gamma t}
$$

The Lotenz-Drude term:
$$
\frac{A_0}{\omega_0^2-\omega^2-i \gamma \omega}
$$

Total dielectric function:
$$
\varepsilon(\omega) = \varepsilon_\infty + \sum \limits_i \frac{A_i}{\omega_i^2-\omega^2-i \gamma_i \omega}
$$
Note: Drude model is a special case of the Lotenz term with $A_0 = \omega_p^2$ and $\omega_0 = 0$




## Lorentzian lineshape (Cauchy distribution)

A random vector $X=(X_1, \ldots, X_k)^T$ is said to have the multivariate Cauchy distribution if every linear combination of its components $Y=a_1X_1+ \cdots + a_kX_k$ has a Cauchy distribution. That is, for any constant vector $a\in \mathbb R^k$, the random variable $Y=a^TX$ should have a univariate Cauchy distribution. The characteristic function of a multivariate Cauchy distribution is given by:

$$
\varphi_X(t) =  e^{ix_0(t)-\gamma(t)}, \!
$$

where $x_0(t$ and $\gamma(t$ are real functions with $x_0(t$ a homogeneous function of degree one and $\gamma(t$ a positive homogeneous function of degree one.

$$
x_0(at) = ax_0(t),
$$
$$
\gamma (at) = |a|\gamma (t),
$$

for all $t$.

An example of a bivariate Cauchy distribution can be given by:
$$
f(x, y; x_0,y_0,\gamma)= { 1 \over 2 \pi } \left[ { \gamma \over ((x - x_0)^2 + (y - y_0)^2 +\gamma^2)^{3/2}  } \right] .
$$
Note that in this example, even though there is no analogue to a covariance matrix, $x$ and $y$ are not Independence (probability theory)|statistically independent.

We also can write this formula for complex variable. Then the probability density function of complex cauchy is :

$$
f(z; z_0,\gamma)= { 1 \over 2 \pi } \left[ { \gamma \over (|z-z_0|^2 +\gamma^2)^{3/2}  } \right] .
$$

Analogous to the univariate density, the multidimensional Cauchy density also relates to the multivariate Student distribution. They are equivalent when the degrees of freedom parameter is equal to one. The density of a $k$ dimension Student distribution with one degree of freedom becomes:

$$
f({\mathbf x}; {\mathbf\mu},{\mathbf\Sigma}, k)= \frac{\Gamma\left(\frac{1+k}{2}\right)}{\Gamma(\frac{1}{2})\pi^{\frac{k}{2}}\left|{\mathbf\Sigma}\right|^{\frac{1}{2}}\left[1+({\mathbf x}-{\mathbf\mu})^T{\mathbf\Sigma}^{-1}({\mathbf x}-{\mathbf\mu})\right]^{\frac{1+k}{2}}} .
$$

Properties and details for this density can be obtained by taking it as a particular case of the multivariate Student density.

## Lorentzian

$$
f(x; x_0,\gamma)= { 1 \over 2 \pi } \left[ { \frac{\gamma}{|x-x_0|^2 +\gamma^2 }} \right]
$$
$$
\varphi(\xi; x_0, \gamma) =  e^{i x_0 \xi} \cdot e^{- \gamma | \xi |}
$$



In [ ]:

Lorentzian(x, x₀, γ) = 1 / 2π * γ / ((x- x₀)^2 + γ^2)

x₀, γ = 6, .5
X = LinRange(0,10,1001)
plot(X, Lorentzian.(X, x₀, γ))


In [ ]:
phi(ξ, x₀, γ) = exp(- im * x₀ * ξ) * exp(-γ * abs(ξ))

Ξ = LinRange(-2π, 2π, 1001)
r = phi.(Ξ, x₀, γ)
plot(Ξ, real(r), imag(r), xlabel="ω", ylabel="Re", zlabel="Im")



# $`@example 1
# plot(
#     plot(Ξ, abs.(r), label="abs"),
#     plot(Ξ, angle.(r), label="angle"),
#     layout=(2,1)
# )
# $`


In [ ]:

Loretnz(omega, A_0, omega_0, gamma) =  A_0 / (omega_0^2 - omega^2 - im * gamma * omega)

omega = 0:.01:10

A_0, omega_0 = 10, 4
gamma = [0 .25 .5 1. 2]

ε = @. Loretnz(omega, A_0, omega_0, gamma)

plot(
    plot(omega, real(ε),label=["γ = $g" for g in gamma], ylims=[-6, 6]),
    plot(omega, imag(ε), label=["γ = $g" for g in gamma]),
    layout=(2,1)
)




In [ ]:

t = 0:0.01:10

x_0, omega_0 = 1, 1
gamma = [0 .25 .5 1. 2]

x(t, x_0, omega_0, gamma) = x_0 * exp(-im * omega_0 * t)* exp(-gamma * t)

X = @. x(t, x_0, omega_0, gamma )
plot(t, real(X))

